# Multiple Task Pneumonia & TB

In [1]:
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3999)])
 

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))

print(('\nYour devices that are available:\n{0}').format(
    [device.name for device in tf.config.experimental.list_physical_devices()]
))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is your GPU available for use?
Yes, your GPU is available: True

Your devices that are available:
['/physical_device:CPU:0', '/physical_device:XLA_CPU:0', '/physical_device:GPU:0', '/physical_device:XLA_GPU:0']


Load and prepare data

In [3]:
import cv2 as cv
import pickle
import random
import numpy as np
 

In [28]:
tb1_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/tb1-256-grayscale", "rb" ) )

p_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/pneumonia-256-grayscale", "rb" ) )
 
# duplicate to have equal amounts
# for i in range(3):
#     tb1_data = tb1_data + tb1_data

x_data = []
y_data_tb1 = []
y_data_p = []
indices_map = [] # true is tb1

for v in tb1_data:
    x_data.append( v[0] )
    y_data_tb1.append(v[1])
    y_data_p.append(0)
    indices_map.append(1)
    

for v in p_data:
    x_data.append( v[0] )
    y_data_tb1.append(0)
    y_data_p.append(v[1])
    indices_map.append(0)

x_data = np.array(x_data)
y_data_tb1 = np.array(y_data_tb1)
y_data_p = np.array(y_data_p)
indices_map = np.array(indices_map)

s = np.arange(len(x_data))
np.random.shuffle(s)

 
x_data = x_data[s]
y_data_tb1 = y_data_tb1[s]
y_data_p = y_data_p[s]


fraction_train = 0.8
train_size = int(len(x_data) * fraction_train)

x_train = x_data[0: train_size]/ 255.0
y_train_tb1 = y_data_tb1[0: train_size]
y_train_p = y_data_p[0: train_size]



x_test = np.array(x_data[train_size:]) / 255.0
y_test_tb1 = np.array(y_data_tb1[train_size:])
y_test_p = np.array(y_data_p[train_size:])


# which index are part of tb1
indices_test_map = np.array( indices_map[s] )[train_size:]
indices_test_tb1 = np.argwhere( indices_test_map == 1 ).flatten()
indices_test_p = np.argwhere( indices_test_map == 0 ).flatten()


# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


print("Train size: {}, test size: {}".format(len(x_train), len(x_test)))

print("tb1_data size: {}, p_data size: {}".format(len(tb1_data), len(p_data)))


Train size: 5214, test size: 1304
tb1_data size: 662, p_data size: 5856


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


tf.keras.backend.clear_session()

model_input = Input(shape=(256, 256, 1))
shared = Conv2D(32, 3, activation='relu')(model_input)
shared = Flatten()(shared)
y1 = Dense(64, activation='relu')(shared)
y1 = Dense(2, name='tb1')(y1)

y2 = Dense(64, activation='relu')(shared)
y2 = Dense(2, name='pneumonia')(y2)

model = Model(inputs=model_input, outputs=[y1, y2])

loss_list =[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)]
test_metrics = {'tb1': 'accuracy', 'pneumonia': 'accuracy' }
model.compile(loss=loss_list,optimizer='adam',metrics=test_metrics)


model.fit(x_train, [y_train_tb1, y_train_p], batch_size=8, epochs=30)



print("Evaluation: ")
print(model.metrics_names)

model.evaluate(x_test, [y_test_tb1, y_test_p], batch_size=8)


                                

Epoch 1/30
652/652 [==============================] - 64s 98ms/step - loss: 1.5957 - tb1_accuracy: 0.9593 - pneumonia_accuracy: 0.7618 - pneumonia_loss: 0.9309 - tb1_loss: 0.6648
Epoch 2/30
652/652 [==============================] - 64s 98ms/step - loss: 0.3065 - tb1_accuracy: 0.9772 - pneumonia_accuracy: 0.9390 - pneumonia_loss: 0.2432 - tb1_loss: 0.0633
Epoch 3/30
652/652 [==============================] - 64s 98ms/step - loss: 0.2030 - tb1_accuracy: 0.9841 - pneumonia_accuracy: 0.9620 - pneumonia_loss: 0.1601 - tb1_loss: 0.0428
Epoch 4/30
652/652 [==============================] - 64s 98ms/step - loss: 0.1466 - tb1_accuracy: 0.9895 - pneumonia_accuracy: 0.9724 - pneumonia_loss: 0.1153 - tb1_loss: 0.0313
Epoch 5/30
652/652 [==============================] - 64s 98ms/step - loss: 0.1230 - tb1_accuracy: 0.9908 - pneumonia_accuracy: 0.9735 - pneumonia_loss: 0.0967 - tb1_loss: 0.0263
Epoch 6/30
652/652 [==============================] - 64s 98ms/step - loss: 0.1515 - tb1_accuracy: 0.9921

[0.5614321827888489,
 0.23830507695674896,
 0.32312706112861633,
 0.9739263653755188,
 0.946319043636322]

In [30]:
predictions = model.predict(x_test)
predictions_tb1 = np.array(tf.argmax(predictions[0], axis=1))
predictions_p = np.array(tf.argmax(predictions[1], axis=1))

# only look at indices for ones with relevant data  

cm = tf.math.confusion_matrix(y_test_tb1[indices_test_tb1], predictions_tb1[indices_test_tb1], 2)
print(cm)

cm = tf.math.confusion_matrix(y_test_p[indices_test_p], predictions_p[indices_test_p], 2)
print(cm)

tf.Tensor(
[[68  1]
 [ 2 77]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[334   4]
 [  6 812]], shape=(2, 2), dtype=int32)
